In [14]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_8272392662f948bfb63722433ce82e01.csv"
df=pd.read_csv(url)
df.head()

,ID,Age,Years_at_Employer,Years_at_Address,Income,Credit_Card_Debt,Automobile_Debt,Outcomes
0,1,32.527993,9.388112,0.297587,37843.67902,-3246.656516,-4794.706076,0
1,2,34.578226,11.972864,1.485103,65765.22990,-15597.775660,-17632.168590,1
2,3,37.697188,12.459833,0.085444,61002.28547,-11401.917620,-7910.242809,1
3,4,28.684510,1.387144,1.837598,19952.73494,-1233.378450,-2408.097355,0
4,5,32.614672,7.488821,0.234122,24970.12812,-1135.680543,-397.323189,0


In [3]:
df.shape

(400, 8)

In [4]:
df.groupby("Outcomes").count().T

Outcomes,0,1
ID,300,100
Age,300,100
Years_at_Employer,300,100
Years_at_Address,300,100
Income,300,100
Credit_Card_Debt,300,100
Automobile_Debt,300,100


In [5]:
df_1=df[df["Outcomes"]==1]
df_1=pd.concat([df_1]*3,axis=0)
df_1.shape

(300, 8)

In [7]:
df_0=df[df["Outcomes"]==0]
df=pd.concat([df_1,df_0],axis=0).sample(frac=1)
df.head()

,ID,Age,Years_at_Employer,Years_at_Address,Income,Credit_Card_Debt,Automobile_Debt,Outcomes
324,325,47.256949,22.771527,0.745067,78771.89608,-3112.985684,-6374.777422,0
270,271,23.640547,3.667022,0.790577,19566.64786,-681.129133,-3837.867875,1
339,340,59.199820,18.397602,0.770073,77925.14644,-8883.190960,-15816.709100,0
311,312,25.781183,1.222991,2.012976,19359.09652,-267.878887,501.328445,0
249,250,33.115751,0.837477,0.325215,28569.91169,-3394.938548,-3841.611142,1


In [8]:
X=df.drop(["ID","Outcomes"],axis=1)
X.head()

,Age,Years_at_Employer,Years_at_Address,Income,Credit_Card_Debt,Automobile_Debt
324,47.256949,22.771527,0.745067,78771.89608,-3112.985684,-6374.777422
270,23.640547,3.667022,0.790577,19566.64786,-681.129133,-3837.867875
339,59.199820,18.397602,0.770073,77925.14644,-8883.190960,-15816.709100
311,25.781183,1.222991,2.012976,19359.09652,-267.878887,501.328445
249,33.115751,0.837477,0.325215,28569.91169,-3394.938548,-3841.611142


In [9]:
y=df[["Outcomes"]]
y.head()

,Outcomes
324,0
270,1
339,0
311,0
249,1


In [10]:
scaler=preprocessing.StandardScaler()
names=X.columns
X=scaler.fit_transform(X)
X=pd.DataFrame(X,columns=names)
X.head()

,Age,Years_at_Employer,Years_at_Address,Income,Credit_Card_Debt,Automobile_Debt
0,1.627936,2.261014,-0.100737,0.676660,0.121916,0.077051
1,-1.237990,-0.607834,-0.025233,-0.580642,0.612188,0.380489
2,3.077241,1.604199,-0.059250,0.658678,-1.041380,-1.052290
3,-0.978217,-0.974845,2.002834,-0.585049,0.695501,0.899496
4,-0.088144,-1.032736,-0.797309,-0.389445,0.065074,0.380041


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(480, 6)
(120, 6)


In [16]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [12]:
tree = RandomForestClassifier(class_weight="balanced_subsample",criterion="entropy",random_state=0)
rf_random = RandomizedSearchCV(estimator =tree, param_distributions = random_grid, n_iter = 300, cv = 8, verbose=2, n_jobs = -1)

In [17]:
rf_random.fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))

Fitting 8 folds for each of 300 candidates, totalling 2400 fits
[CV] n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True 


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True, total=   3.7s
[CV] n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s


[CV]  n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True, total=   3.5s
[CV] n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True 
[CV]  n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True, total=   3.9s
[CV] n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True 
[CV]  n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True, total=   3.8s
[CV] n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True 
[CV]  n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True, total=   3.9s
[CV] n_estimators=911, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=80, bootstrap=True 
[CV]  n_estimators=911, min_samples_

...

[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False, total=   1.7s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False, total=   1.5s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False, total=   1.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False, total=   1.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, bootstrap=False 
[CV]  n_estimators=2

[CV]  n_estimators=911, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=   2.1s
[CV] n_estimators=911, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=911, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=   6.6s
[CV] n_estimators=911, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=911, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=   7.1s
[CV] n_estimators=911, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=911, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=   7.0s
[CV] n_estimators=911, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=911, min_

[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=100, bootstrap=False, total=   2.2s
[CV] n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=100, bootstrap=False 
[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=100, bootstrap=False, total=   2.3s
[CV] n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=100, bootstrap=False 
[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=100, bootstrap=False, total=   2.3s
[CV] n_estimators=288, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=True 
[CV]  n_estimators=288, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=True, total=   0.7s
[CV] n_estimators=288, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=True 
[CV]  n_estimators

[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed: 85.6min finished


RandomizedSearchCV(cv=8,
                   estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                                    criterion='entropy',
                                                    random_state=0),
                   n_iter=300, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 288, 377, 466,
                                                         555, 644, 733, 822,
                                                 

In [18]:
rf_random.best_params_

{'n_estimators': 377,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': False}

In [19]:
best_random = rf_random.best_estimator_
y_pred=best_random.predict(X_test)
print("Accuracy on test data : ",accuracy_score(y_test,y_pred))

Accuracy on test data :  0.95


In [20]:
from sklearn.metrics import roc_auc_score
print("AUC on test data: ",roc_auc_score(y_test,y_pred))

AUC on test data:  0.9500000000000001
